In [2]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential, Model, model_from_json
from keras.layers import Conv2D, Activation, MaxPool2D, Dropout, Dense, BatchNormalization, Flatten
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [3]:
ds = pd.read_csv("fer2013.csv")
ds.head()
#0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
ds.tail()

,emotion,pixels,Usage
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest
35886,2,19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...,PrivateTest


In [5]:
ds["Usage"].value_counts()

Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64

In [4]:
train = ds[["emotion", "pixels"]][ds["Usage"] == "Training"]
train['pixels'] = train['pixels'].apply(lambda x: np.fromstring(x, sep=' '))
train_pix = np.vstack(train['pixels'].values)
test = ds[["emotion", "pixels"]][ds["Usage"] == "PublicTest"]
test['pixels'] = test['pixels'].apply(lambda x: np.fromstring(x, sep=' '))
test_pix = np.vstack(test['pixels'].values)
train_pix.shape, test_pix.shape

((28709, 2304), (3589, 2304))

In [5]:
#0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
#0=angry, 1=disgust+fear+surprise, 3=Happy, 4=sad, 6= neutral
#apending 2 and 5 to 1; 5 to 2; 6 to 4
#:::: set append = false to avoid changes
def e_ind(x):
    if(x==2 or x==5):
        return 1
    elif(x==5):
        return 2
    elif(x==6):
        return 4
    else:
        return x
F_S_D = True
if(F_S_D):
    train['emotion'] = train['emotion'].apply(lambda x: e_ind(x))
    test['emotion'] = test['emotion'].apply(lambda x: e_ind(x))
train_ind = np.array(train["emotion"])
test_ind = np.array(test["emotion"])
train_ind.shape, test_ind.shape

((28709,), (3589,))

In [6]:
train_pix = train_pix.reshape(-1,48,48,1)
train_ind = np_utils.to_categorical(train_ind)
test_pix = test_pix.reshape(-1,48,48,1)
test_ind = np_utils.to_categorical(test_ind)
train_ind.shape, test_ind.shape
#train_pix.shape, test_pix.shape

((28709, 5), (3589, 5))

In [7]:
model = Sequential()

# 1st stage
model.add(Conv2D(32, 3, input_shape=(48, 48, 1), padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, 3, padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

# 2nd stage
model.add(Conv2D(64, 3, padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, 3, padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.25))

# 3rd stage
model.add(Conv2D(128, 3, padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, 3, padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.25))

# FC layers
model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(7))
model.add(Activation('softmax'))
if(F_S_D):
    model.add(Dense(5))
else:
    model.add(Dense(7))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
batch_normalization_3 (Batch (None, 24, 24, 64)        256       
__________

In [9]:
checkPoint = ModelCheckpoint(filepath='chkPt3.h5', verbose=1, save_best_only=True)

res = model.fit(train_pix, train_ind, epochs=10,
                 shuffle=True,
                 batch_size=100, 
                 validation_data=(test_pix, test_ind),
                 callbacks=[checkPoint], 
                 verbose=2)

# save model to json
mod1_json = model.to_json()
with open("model3.json", "w") as json_file:
    json_file.write(mod1_json)

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
 - 1521s - loss: 1.0232 - acc: 0.6156 - val_loss: 0.9747 - val_acc: 0.6381

Epoch 00001: val_loss improved from inf to 0.97472, saving model to chkPt3.h5
Epoch 2/10
 - 1511s - loss: 0.9535 - acc: 0.6481 - val_loss: 0.9559 - val_acc: 0.6431

Epoch 00002: val_loss improved from 0.97472 to 0.95590, saving model to chkPt3.h5
Epoch 3/10
 - 1509s - loss: 0.9188 - acc: 0.6587 - val_loss: 0.9838 - val_acc: 0.6283

Epoch 00003: val_loss did not improve from 0.95590
Epoch 4/10
 - 1512s - loss: 0.8836 - acc: 0.6739 - val_loss: 0.9035 - val_acc: 0.6626

Epoch 00004: val_loss improved from 0.95590 to 0.90349, saving model to chkPt3.h5
Epoch 5/10
 - 1516s - loss: 0.8527 - acc: 0.6842 - val_loss: 0.8802 - val_acc: 0.6712

Epoch 00005: val_loss improved from 0.90349 to 0.88020, saving model to chkPt3.h5
Epoch 6/10
 - 1678s - loss: 0.8236 - acc: 0.6984 - val_loss: 0.9045 - val_acc: 0.6606

Epoch 00006: val_loss did not improve from 0.88020
Epo